In [1]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
import mindpype as mp
import pkgutil, numpy as np

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline.

In [3]:
# Create the pipeline using the factory method
session = mp.Session.create()

Following the creation of the pipeline, we need to create the graphs, which are the representations of the pipeline in MindPype.

Each distinct pipeline would require a different graph object. For this tutorial, we will only require a single graph.

In [4]:
# Create the graph using the factory method
graph = mp.Graph.create(session)

With a graph created, we can now attach our input and output data containers. Since the input data is known, we can create a tensor (the data container in MindPype), using the ```create_from_data()``` factory method. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.       

In [5]:
# Create a random input tensor using the factory method
input_tensor = mp.Tensor.create_from_data(session, data=np.random.rand(10, 10))

# Create an output tensor using the factory method
output_tensor = mp.Tensor.create(session, shape=(10, 10))

With the input and output tensors, we can know add the processing nodes to the pipeline. For this example, we are using a single processing step, or kernel; the baseline correction kernel. We can use the following factory method to add the kernel to a node, and add the node to the graph we created previously.

Futhermore, we must pass the input and output tensors, as well as the required parameters for the kernel (in this case, the baseline period), to the factory method shown below.

In [6]:
# Create the baseline node, passing the graph, input tensor, output tensor, and baseline period
baseline_node = mp.kernels.BaselineCorrectionKernel.add_to_graph(graph, input_tensor, output_tensor, baseline_period = [0, 10])

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

In [7]:
# Verify the structure and inputs/outputs of the graph
graph.verify()


Verifying kernel BaselineCorrection...


Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.

In [8]:
# Initialize the graph - optional for this pipeline
graph.initialize()

The graph is now ready to execute the provided input data. For other paradigms, we may opt to execute the graph in a loop. If we were using class-separated data, we would need to pass a task label to poll the correct task.

In [9]:
# Execute the graph
graph.execute()



Executing trial with label: None


In [10]:
# Visualize the input and output data
print(input_tensor.data)
print(output_tensor.data)

[[0.14070506 0.62204865 0.98030605 0.58592992 0.33121202 0.48122253
  0.88649898 0.52962295 0.41105905 0.05868479]
 [0.79552362 0.49093481 0.02812619 0.22394131 0.26579121 0.22808016
  0.73185244 0.80335125 0.15158811 0.2957604 ]
 [0.94232653 0.07572276 0.49243867 0.72549448 0.8612449  0.62907257
  0.39313045 0.84703263 0.32474972 0.80671659]
 [0.32303957 0.44578549 0.05388623 0.68170718 0.60506163 0.98016904
  0.99979419 0.7838448  0.84605226 0.11806063]
 [0.46373369 0.83275182 0.72886362 0.99624551 0.44166519 0.66733276
  0.89452012 0.26031007 0.56338177 0.45011888]
 [0.47210486 0.75716526 0.56751047 0.98187012 0.46743085 0.87182571
  0.10147679 0.41051062 0.32632948 0.97615911]
 [0.13913958 0.86313918 0.26025287 0.89463798 0.91449125 0.21363421
  0.03509049 0.55724349 0.73230261 0.3596177 ]
 [0.1475992  0.39332856 0.73696375 0.26188794 0.32611825 0.99965466
  0.49334128 0.72389672 0.11845279 0.01710657]
 [0.23739472 0.75699878 0.53378766 0.29446468 0.62965647 0.76935597
  0.52638409

In [11]:
# Confirm proper functionality
corrected = input_tensor.data - np.mean(input_tensor.data, axis = -1, keepdims = True)

assert np.allclose(output_tensor.data, corrected)